In [ ]:
import json
import re
from collections import Counter
from datetime import datetime

class TelegramProAnalyzer:
    def __init__(self):
        self.messages_dict = {}  
        self.users_stats = {}    
        self.interactions = {}   
        self.all_messages_list = []
        self.hourly_activity = Counter() 
        self.my_accounts = ["لؤي الرازي", "الُحـ๛ـــــآمٌ", "قناة الُحـ๛ـــــآمٌ الرئيسية"]

    def load_and_clean(self, file_paths):
        for file_path in file_paths:
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    for msg in data.get('messages', []):
                        self.messages_dict[msg.get('id')] = msg
                        self.all_messages_list.append(msg)
                print(f"✅ تم تحميل: {file_path}")
            except Exception as e:
                print(f"❌ خطأ في التحميل: {e}")

    def _extract_text(self, msg):
        text_data = msg.get('text', '')
        if isinstance(text_data, list):
            return "".join([p if isinstance(p, str) else p.get('text', '') for p in text_data])
        return str(text_data)

    def process_data(self):
        self.all_messages_list.sort(key=lambda x: x.get('date', ''))
        for msg in self.all_messages_list:
            raw_name = msg.get('from')
            sender_name = str(raw_name) if raw_name is not None else "حساب محذوف"
            
            if any(acc in sender_name for acc in self.my_accounts):
                display_name = "أنا (الحسابات المدمجة)"
                sender_id = "unified_me_id"
            else:
                display_name = sender_name
                sender_id = msg.get('from_id', 'unknown_id')

            text = self._extract_text(msg)
            date_str = msg.get('date', '')
            
            if date_str:
                dt = datetime.fromisoformat(date_str)
                self.hourly_activity[dt.hour] += 1

            if sender_id and text.strip():
                if sender_id not in self.users_stats:
                    self.users_stats[sender_id] = {
                        'name': display_name, 'msg_count': 0, 
                        'words_count': 0, 'replies_sent': 0, 
                        'received_replies': 0, 'all_text': ""
                    }
                
                stats = self.users_stats[sender_id]
                stats['msg_count'] += 1
                stats['words_count'] += len(text.split())
                stats['all_text'] += " " + text.lower()

                reply_to = msg.get('reply_to_message_id')
                if reply_to:
                    stats['replies_sent'] += 1
                    original_msg = self.messages_dict.get(reply_to)
                    if original_msg:
                        t_raw = original_msg.get('from')
                        t_name = str(t_raw) if t_raw is not None else "حساب محذوف"
                        t_id = "unified_me_id" if any(acc in t_name for acc in self.my_accounts) else original_msg.get('from_id')
                        if t_id in self.users_stats:
                            self.users_stats[t_id]['received_replies'] += 1
                            pair = tuple(sorted([str(sender_id), str(t_id)]))
                            self.interactions[pair] = self.interactions.get(pair, 0) + 1

    def generate_user_persona(self, target_id):
        """إنشاء بروفايل وصفي شامل لكل عضو"""
        s = self.users_stats.get(target_id)
        if not s: return "العضو غير موجود"
        avg_len = s['words_count'] / s['msg_count']
        res_rate = (s['replies_sent'] / s['msg_count']) * 100
        
        persona = [
            f"👤 البطاقة الشخصية: {s['name']}",
            f"   • النمط: {'تفصيلي' if avg_len > 10 else 'مقتضب'}",
            f"   • التفاعل: {res_rate:.1f}%",
            f"   • النشاط: {s['msg_count']} رسالة"
        ]
        return "\n".join(persona)

    def ask_the_data(self, query):
        """محرك الإجابة على الأسئلة عبر البحث في التاريخ"""
        results = []
        keywords = [w for w in query.split() if len(w) > 2]
        for msg in self.all_messages_list[-5000:]: # البحث في آخر 5000 رسالة للسرعة
            text = self._extract_text(msg)
            if any(key in text for key in keywords):
                results.append(f"📅 {msg.get('date','')[:10]} | {msg.get('from')}: {text[:100]}")
                if len(results) > 3: break
        return "\n".join(results) if results else "لم أجد إجابة."